# 5차시: 외부 연동 & 실시간 데이터 처리

## 학습 목표
- OpenAI Function Calling 이해 및 활용
- 외부 API 연동 (날씨, 검색 등)
- 데이터베이스 연동 및 실시간 조회
- Tool 시스템 구축 및 에러 핸들링
- Circuit Breaker 패턴 적용

## 주요 구현 내용
1. **Tool Calling 시스템**: OpenAI Function Calling 구현
2. **외부 API 연동**: 날씨 API, 검색 API 연결
3. **데이터베이스 연동**: SQLite/PostgreSQL 실시간 조회
4. **에러 핸들링**: Circuit Breaker, 재시도 로직

## 1. 환경 설정 및 기본 구조

In [ ]:
#!/usr/bin/env python3
import os
import sys
import json
import time
import uuid
import logging
import sqlite3
import requests
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
from datetime import datetime
from abc import ABC, abstractmethod
from functools import wraps

# 외부 라이브러리
from openai import OpenAI
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# 로컬 설정
sys.path.append('..')
from config import get_config

config = get_config()
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 환경 설정 완료")
print(f"🤖 OpenAI 모델: {config.llm.openai_model}")
print(f"🔧 Function Calling 지원: {'✅' if 'gpt-' in config.llm.openai_model else '❌'}")

## 2. 데이터 구조 및 에러 핸들링

Tool 실행 결과와 Circuit Breaker 패턴을 구현합니다.

In [ ]:
@dataclass
class ToolResult:
    """Tool 실행 결과"""
    success: bool
    data: Any
    error: Optional[str] = None
    execution_time: float = 0.0
    metadata: Dict[str, Any] = None
    
    def __post_init__(self):
        if self.metadata is None:
            self.metadata = {}

# Circuit Breaker 패턴 구현
class CircuitBreakerState:
    """Circuit Breaker 상태"""
    CLOSED = "closed"       # 정상 상태
    OPEN = "open"           # 차단 상태 (에러 다발)
    HALF_OPEN = "half_open" # 반개방 상태 (복구 테스트)

class CircuitBreaker:
    """Circuit Breaker 패턴으로 외부 서비스 호출 보호"""
    
    def __init__(self, failure_threshold: int = 3, recovery_timeout: int = 30):
        self.failure_threshold = failure_threshold
        self.recovery_timeout = recovery_timeout
        
        # 상태 관리
        self.failure_count = 0
        self.last_failure_time = None
        self.state = CircuitBreakerState.CLOSED
        
        print(f"🔧 Circuit Breaker 초기화 - 실패 임계값: {failure_threshold}, 복구 시간: {recovery_timeout}초")
    
    def call(self, func, *args, **kwargs):
        """함수 호출 with Circuit Breaker"""
        if self.state == CircuitBreakerState.OPEN:
            # 복구 시간 확인
            if self._should_attempt_reset():
                self.state = CircuitBreakerState.HALF_OPEN
                print("🔄 Circuit Breaker: HALF_OPEN 상태로 전환")
            else:
                raise Exception(f"Circuit Breaker OPEN - 서비스 일시적으로 사용 불가")
        
        try:
            # 함수 실행
            result = func(*args, **kwargs)
            self._on_success()
            return result
        
        except Exception as e:
            self._on_failure()
            raise e
    
    def _should_attempt_reset(self) -> bool:
        """복구 시도 여부 확인"""
        return (
            self.last_failure_time and 
            time.time() - self.last_failure_time >= self.recovery_timeout
        )
    
    def _on_success(self):
        """성공 시 상태 관리"""
        self.failure_count = 0
        if self.state == CircuitBreakerState.HALF_OPEN:
            self.state = CircuitBreakerState.CLOSED
            print("✅ Circuit Breaker: CLOSED 상태로 복구")
    
    def _on_failure(self):
        """실패 시 상태 관리"""
        self.failure_count += 1
        self.last_failure_time = time.time()
        
        if self.failure_count >= self.failure_threshold:
            self.state = CircuitBreakerState.OPEN
            print(f"🚨 Circuit Breaker: OPEN 상태로 전환 ({self.failure_count}회 연속 실패)")

# 재시도 데코레이터
def retry_on_failure(max_attempts: int = 3, delay: float = 1.0):
    """재시도 데코레이터"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(max_attempts):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt < max_attempts - 1:
                        print(f"⚠️ 재시도 {attempt + 1}/{max_attempts}: {str(e)}")
                        time.sleep(delay)
                    else:
                        print(f"❌ 최종 실패: {str(e)}")
                        raise e
        return wrapper
    return decorator

# Circuit Breaker 테스트
def test_circuit_breaker():
    """Circuit Breaker 패턴 테스트"""
    cb = CircuitBreaker(failure_threshold=3, recovery_timeout=5)
    
    def unreliable_function(should_fail=True):
        if should_fail:
            raise Exception("서비스 오류")
        return "성공!"
    
    print("\n🧪 Circuit Breaker 테스트:")
    
    # 실패 시뮬레이션
    for i in range(5):
        try:
            result = cb.call(unreliable_function, should_fail=True)
            print(f"  {i+1}. {result}")
        except Exception as e:
            print(f"  {i+1}. 실패: {e}")
    
    # 복구 테스트
    print("\n⏳ 5초 대기 후 복구 테스트...")
    time.sleep(6)
    
    try:
        result = cb.call(unreliable_function, should_fail=False)
        print(f"  복구 후: {result}")
    except Exception as e:
        print(f"  복구 실패: {e}")

# 테스트 실행
test_circuit_breaker()

## 3. Tool 베이스 클래스 및 날씨 API 연동

모든 Tool의 기본이 되는 베이스 클래스와 날씨 조회 도구를 구현합니다.

In [ ]:
# Tool 베이스 클래스
class BaseTool(ABC):
    """Tool 베이스 클래스"""
    
    def __init__(self, name: str, description: str, parameters: Dict[str, Any]):
        self.name = name
        self.description = description
        self.parameters = parameters
        self.circuit_breaker = CircuitBreaker()
        
    @abstractmethod
    def execute(self, **kwargs) -> ToolResult:
        """Tool 실행 (추상 메서드)"""
        pass
    
    def get_openai_function_spec(self) -> Dict[str, Any]:
        """OpenAI Function Calling용 스펙 반환"""
        return {
            "name": self.name,
            "description": self.description,
            "parameters": self.parameters
        }
    
    def _create_result(self, success: bool, data: Any = None, 
                      error: str = None, execution_time: float = 0.0) -> ToolResult:
        """ToolResult 생성 헬퍼"""
        return ToolResult(
            success=success,
            data=data,
            error=error,
            execution_time=execution_time,
            metadata={
                'tool_name': self.name,
                'timestamp': datetime.now().isoformat()
            }
        )

# 날씨 조회 Tool
class WeatherTool(BaseTool):
    """날씨 정보 조회 도구"""
    
    def __init__(self, api_key: str = None):
        super().__init__(
            name="get_weather",
            description="특정 도시의 현재 날씨 정보를 조회합니다",
            parameters={
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "날씨를 조회할 도시명 (예: Seoul, Tokyo, New York)"
                    },
                    "units": {
                        "type": "string",
                        "enum": ["metric", "imperial", "kelvin"],
                        "description": "온도 단위 (metric=섭씨, imperial=화씨)",
                        "default": "metric"
                    }
                },
                "required": ["city"]
            }
        )
        
        # API 키 설정 (실제 환경에서는 환경변수에서 가져오기)
        self.api_key = api_key or "demo_api_key"
        self.base_url = "http://api.openweathermap.org/data/2.5/weather"
        
        # HTTP 세션 설정
        self.session = requests.Session()
        retry_strategy = Retry(
            total=3,
            status_forcelist=[429, 500, 502, 503, 504],
            method_whitelist=["HEAD", "GET", "OPTIONS"],
            backoff_factor=1
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        self.session.mount("http://", adapter)
        self.session.mount("https://", adapter)
        
        print("🌤️ 날씨 조회 도구 초기화 완료")
    
    @retry_on_failure(max_attempts=3, delay=1.0)
    def execute(self, city: str, units: str = "metric") -> ToolResult:
        """날씨 정보 조회 실행"""
        start_time = time.time()
        print(f"🌤️ 날씨 조회 시작: {city} ({units})")
        
        try:
            # 실제 API 키가 없는 경우 모의 데이터 반환
            if self.api_key == "demo_api_key":
                print("ℹ️ 데모 모드: 모의 날씨 데이터 반환")
                weather_data = self._get_mock_weather_data(city, units)
            else:
                # 실제 API 호출
                def api_call():
                    params = {
                        'q': city,
                        'appid': self.api_key,
                        'units': units,
                        'lang': 'kr'
                    }
                    response = self.session.get(self.base_url, params=params, timeout=10)
                    response.raise_for_status()
                    return response.json()
                
                weather_data = self.circuit_breaker.call(api_call)
            
            # 데이터 파싱
            result_data = self._parse_weather_data(weather_data, units)
            execution_time = time.time() - start_time
            
            print(f"✅ 날씨 조회 완료: {result_data['temperature']}{result_data['unit']}")
            
            return self._create_result(
                success=True,
                data=result_data,
                execution_time=execution_time
            )
            
        except Exception as e:
            execution_time = time.time() - start_time
            error_msg = f"날씨 조회 실패: {str(e)}"
            print(f"❌ {error_msg}")
            
            return self._create_result(
                success=False,
                error=error_msg,
                execution_time=execution_time
            )
    
    def _get_mock_weather_data(self, city: str, units: str) -> Dict[str, Any]:
        """모의 날씨 데이터 생성"""
        import random
        
        # 도시별 특성화된 날씨 데이터
        city_weather = {
            'seoul': {'temp_range': (15, 30), 'weather': 'Clear', 'desc': '맑음'},
            'tokyo': {'temp_range': (18, 32), 'weather': 'Clouds', 'desc': '구름많음'},
            'new york': {'temp_range': (20, 35), 'weather': 'Rain', 'desc': '비'},
            'london': {'temp_range': (10, 20), 'weather': 'Clouds', 'desc': '흐림'}
        }
        
        city_lower = city.lower()
        weather_info = city_weather.get(city_lower, {
            'temp_range': (15, 25), 'weather': 'Clear', 'desc': '맑음'
        })
        
        temp = random.randint(*weather_info['temp_range'])
        if units == "imperial":
            temp = temp * 9/5 + 32
        elif units == "kelvin":
            temp = temp + 273.15
        
        mock_data = {
            "weather": [{"main": weather_info['weather'], "description": weather_info['desc']}],
            "main": {
                "temp": temp,
                "feels_like": temp + random.randint(-3, 3),
                "humidity": random.randint(40, 80),
                "pressure": random.randint(1000, 1020)
            },
            "wind": {"speed": random.randint(1, 10)},
            "name": city
        }
        return mock_data
    
    def _parse_weather_data(self, data: Dict[str, Any], units: str) -> Dict[str, Any]:
        """날씨 데이터 파싱"""
        unit_symbol = {"metric": "°C", "imperial": "°F", "kelvin": "K"}[units]
        
        return {
            "city": data.get("name", "Unknown"),
            "temperature": round(data["main"]["temp"], 1),
            "feels_like": round(data["main"]["feels_like"], 1),
            "description": data["weather"][0]["description"],
            "humidity": data["main"]["humidity"],
            "pressure": data["main"]["pressure"],
            "wind_speed": data["wind"]["speed"],
            "unit": unit_symbol
        }

# 날씨 도구 테스트
def test_weather_tool():
    """날씨 도구 테스트"""
    print("\n🧪 날씨 도구 테스트:")
    
    weather_tool = WeatherTool()
    
    # OpenAI Function 스펙 확인
    function_spec = weather_tool.get_openai_function_spec()
    print(f"\n📋 Function 스펙:")
    print(f"  이름: {function_spec['name']}")
    print(f"  설명: {function_spec['description']}")
    
    # 여러 도시 테스트
    test_cities = ["Seoul", "Tokyo", "New York", "London"]
    
    for city in test_cities:
        print(f"\n🌍 {city} 날씨 조회:")
        result = weather_tool.execute(city=city, units="metric")
        
        if result.success:
            data = result.data
            print(f"  🌡️ 온도: {data['temperature']}{data['unit']}")
            print(f"  🌤️ 날씨: {data['description']}")
            print(f"  💨 습도: {data['humidity']}%")
            print(f"  ⏱️ 실행시간: {result.execution_time:.2f}초")
        else:
            print(f"  ❌ 오류: {result.error}")

# 테스트 실행
test_weather_tool()

## 4. 웹 검색 Tool 구현

인터넷 검색 기능을 제공하는 도구를 구현합니다.

In [ ]:
# 검색 Tool
class SearchTool(BaseTool):
    """웹 검색 도구"""
    
    def __init__(self, api_key: str = None):
        super().__init__(
            name="web_search",
            description="인터넷에서 정보를 검색합니다",
            parameters={
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "검색할 키워드 또는 질문"
                    },
                    "num_results": {
                        "type": "integer",
                        "description": "반환할 검색 결과 수 (기본값: 5)",
                        "minimum": 1,
                        "maximum": 10,
                        "default": 5
                    }
                },
                "required": ["query"]
            }
        )
        
        self.api_key = api_key or "demo_search_key"
        
        print("🔍 검색 도구 초기화 완료")
    
    def execute(self, query: str, num_results: int = 5) -> ToolResult:
        """웹 검색 실행"""
        start_time = time.time()
        print(f"🔍 웹 검색 시작: '{query}' (결과 수: {num_results})")
        
        try:
            # 실제 API 키가 없는 경우 모의 데이터 반환
            if self.api_key == "demo_search_key":
                print("ℹ️ 데모 모드: 모의 검색 결과 반환")
                search_results = self._get_mock_search_results(query, num_results)
            else:
                search_results = self._perform_real_search(query, num_results)
            
            execution_time = time.time() - start_time
            
            print(f"✅ 웹 검색 완료: {len(search_results)}개 결과 반환")
            
            return self._create_result(
                success=True,
                data={
                    "query": query,
                    "results": search_results,
                    "result_count": len(search_results)
                },
                execution_time=execution_time
            )
            
        except Exception as e:
            execution_time = time.time() - start_time
            error_msg = f"웹 검색 실패: {str(e)}"
            print(f"❌ {error_msg}")
            
            return self._create_result(
                success=False,
                error=error_msg,
                execution_time=execution_time
            )
    
    def _get_mock_search_results(self, query: str, num_results: int) -> List[Dict[str, Any]]:
        """모의 검색 결과 생성"""
        
        # 쿼리별 맞춤형 결과 생성
        query_templates = {
            'python': [
                ('Python 공식 문서', 'https://docs.python.org/', 'Python의 공식 문서입니다. 튜토리얼, 라이브러리 참조, 언어 레퍼런스가 포함되어 있습니다.'),
                ('Python 다운로드', 'https://www.python.org/downloads/', 'Python의 최신 버전을 다운로드할 수 있습니다.'),
                ('Python 패키지 저장소 - PyPI', 'https://pypi.org/', 'Python 패키지를 찾고 설치할 수 있는 공식 저장소입니다.')
            ],
            'ai': [
                ('OpenAI', 'https://openai.com/', 'ChatGPT와 GPT 모델을 개발한 AI 연구소입니다.'),
                ('AI 뉴스 - MIT Technology Review', 'https://www.technologyreview.com/topic/artificial-intelligence/', 'AI 관련 최신 뉴스와 연구 동향을 제공합니다.'),
                ('Machine Learning 강의 - Coursera', 'https://www.coursera.org/learn/machine-learning', 'Andrew Ng 교수의 유명한 머신러닝 강의입니다.')
            ],
            'chatbot': [
                ('Chatbot 개발 가이드', 'https://example.com/chatbot-guide', '챗봇 개발을 위한 완전한 가이드입니다.'),
                ('OpenAI API 문서', 'https://platform.openai.com/docs/', 'OpenAI API를 사용한 챗봇 개발 문서입니다.'),
                ('LangChain 문서', 'https://langchain.readthedocs.io/', 'LLM 애플리케이션 개발 프레임워크입니다.')
            ]
        }
        
        # 쿼리와 매칭되는 템플릿 찾기
        results = []
        query_lower = query.lower()
        
        for keyword, templates in query_templates.items():
            if keyword in query_lower:
                for i, (title, url, snippet) in enumerate(templates[:num_results]):
                    results.append({
                        'title': title,
                        'url': url,
                        'snippet': snippet,
                        'relevance_score': 0.9 - (i * 0.1)
                    })
                break
        
        # 기본 결과가 없으면 일반적인 결과 생성
        if not results:
            for i in range(num_results):
                results.append({
                    'title': f"{query}에 대한 검색 결과 {i+1}",
                    'url': f"https://example.com/result_{i+1}",
                    'snippet': f"이것은 '{query}'에 대한 {i+1}번째 검색 결과입니다. 유용한 정보를 포함하고 있습니다.",
                    'relevance_score': 0.8 - (i * 0.1)
                })
        
        return results[:num_results]
    
    @retry_on_failure(max_attempts=2)
    def _perform_real_search(self, query: str, num_results: int) -> List[Dict[str, Any]]:
        """실제 검색 API 호출 (DuckDuckGo 예시)"""
        
        # DuckDuckGo instant answer API 사용 (무료)
        url = "https://api.duckduckgo.com/"
        params = {
            'q': query,
            'format': 'json',
            'no_html': '1',
            'skip_disambig': '1'
        }
        
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        
        results = []
        
        # Abstract 결과 추가
        if data.get('Abstract'):
            results.append({
                'title': data.get('Heading', query),
                'url': data.get('AbstractURL', ''),
                'snippet': data.get('Abstract', ''),
                'relevance_score': 1.0
            })
        
        # Related topics에서 추가 결과 가져오기
        for i, topic in enumerate(data.get('RelatedTopics', [])[:num_results-1]):
            if isinstance(topic, dict) and 'Text' in topic:
                results.append({
                    'title': topic.get('Text', '')[:100],
                    'url': topic.get('FirstURL', ''),
                    'snippet': topic.get('Text', ''),
                    'relevance_score': 0.9 - (i * 0.1)
                })
        
        return results[:num_results]

# 검색 도구 테스트
def test_search_tool():
    """검색 도구 테스트"""
    print("\n🧪 검색 도구 테스트:")
    
    search_tool = SearchTool()
    
    # 다양한 쿼리 테스트
    test_queries = [
        ("Python 프로그래밍", 3),
        ("AI 인공지능", 2),
        ("chatbot 개발", 4)
    ]
    
    for query, num_results in test_queries:
        print(f"\n🔍 검색: '{query}' (결과 {num_results}개)")
        result = search_tool.execute(query=query, num_results=num_results)
        
        if result.success:
            data = result.data
            print(f"  📊 총 결과: {data['result_count']}개")
            
            for i, res in enumerate(data['results'], 1):
                print(f"  {i}. {res['title']}")
                print(f"     {res['url']}")
                print(f"     {res['snippet'][:80]}...")
                if 'relevance_score' in res:
                    print(f"     관련도: {res['relevance_score']:.1f}")
                print()
            
            print(f"  ⏱️ 실행시간: {result.execution_time:.2f}초")
        else:
            print(f"  ❌ 오류: {result.error}")

# 테스트 실행
test_search_tool()

## 5. 데이터베이스 연동 Tool 구현

SQLite를 사용한 데이터베이스 조회 도구를 구현합니다.

In [ ]:
# 데이터베이스 Tool
class DatabaseTool(BaseTool):
    """데이터베이스 조회 도구"""
    
    def __init__(self, db_path: str = ":memory:"):
        super().__init__(
            name="query_database",
            description="데이터베이스에서 정보를 조회합니다",
            parameters={
                "type": "object",
                "properties": {
                    "query_type": {
                        "type": "string",
                        "enum": ["user_info", "order_history", "product_info", "analytics"],
                        "description": "조회할 데이터 유형"
                    },
                    "params": {
                        "type": "object",
                        "description": "조회 파라미터 (user_id, product_id 등)",
                        "additionalProperties": True
                    }
                },
                "required": ["query_type"]
            }
        )
        
        self.db_path = db_path
        self.connection = None
        
        self._init_database()
        print(f"💾 데이터베이스 도구 초기화 완료 - 경로: {db_path}")
    
    def _init_database(self):
        """데이터베이스 연결 초기화 및 테스트 데이터 생성"""
        try:
            self.connection = sqlite3.connect(self.db_path, check_same_thread=False)
            self.connection.row_factory = sqlite3.Row
            self._create_test_tables()
            print("✅ 데이터베이스 연결 및 테이블 생성 완료")
            
        except Exception as e:
            print(f"❌ 데이터베이스 초기화 실패: {e}")
            raise
    
    def _create_test_tables(self):
        """테스트용 테이블 및 데이터 생성"""
        cursor = self.connection.cursor()
        
        # 사용자 테이블
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                user_id INTEGER PRIMARY KEY,
                username TEXT UNIQUE,
                email TEXT,
                created_at TEXT,
                last_login TEXT,
                status TEXT DEFAULT 'active'
            )
        ''')
        
        # 상품 테이블
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS products (
                product_id INTEGER PRIMARY KEY,
                name TEXT,
                category TEXT,
                price REAL,
                stock INTEGER,
                description TEXT
            )
        ''')
        
        # 주문 테이블
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS orders (
                order_id INTEGER PRIMARY KEY,
                user_id INTEGER,
                product_id INTEGER,
                quantity INTEGER,
                order_date TEXT,
                status TEXT,
                total_amount REAL,
                FOREIGN KEY (user_id) REFERENCES users(user_id),
                FOREIGN KEY (product_id) REFERENCES products(product_id)
            )
        ''')
        
        # 테스트 데이터 삽입
        test_users = [
            (1, 'john_doe', 'john@example.com', '2024-01-15', '2024-08-30', 'active'),
            (2, 'jane_smith', 'jane@example.com', '2024-02-20', '2024-08-29', 'active'),
            (3, 'bob_wilson', 'bob@example.com', '2024-03-10', '2024-08-28', 'inactive'),
            (4, 'alice_brown', 'alice@example.com', '2024-04-05', '2024-08-27', 'active')
        ]
        
        test_products = [
            (1, 'Python 프로그래밍 책', '도서', 29.99, 50, 'Python 프로그래밍 입문서'),
            (2, '무선 마우스', '전자제품', 45.00, 30, '고품질 무선 마우스'),
            (3, '커피 머그컵', '생활용품', 12.50, 100, '세라믹 커피 머그컵'),
            (4, 'AI 개발자 가이드', '도서', 39.99, 25, 'AI 개발 완전 가이드'),
            (5, '스마트 키보드', '전자제품', 89.99, 15, '백라이트 기계식 키보드')
        ]
        
        test_orders = [
            (1, 1, 1, 2, '2024-08-25', 'completed', 59.98),
            (2, 1, 3, 1, '2024-08-26', 'completed', 12.50),
            (3, 2, 2, 1, '2024-08-27', 'pending', 45.00),
            (4, 3, 1, 1, '2024-08-28', 'completed', 29.99),
            (5, 4, 4, 1, '2024-08-29', 'completed', 39.99),
            (6, 2, 5, 1, '2024-08-30', 'pending', 89.99)
        ]
        
        cursor.executemany('INSERT OR REPLACE INTO users VALUES (?, ?, ?, ?, ?, ?)', test_users)
        cursor.executemany('INSERT OR REPLACE INTO products VALUES (?, ?, ?, ?, ?, ?)', test_products)
        cursor.executemany('INSERT OR REPLACE INTO orders VALUES (?, ?, ?, ?, ?, ?, ?)', test_orders)
        
        self.connection.commit()
        print(f"📊 테스트 데이터 생성 완료 - 사용자: {len(test_users)}, 상품: {len(test_products)}, 주문: {len(test_orders)}")
    
    def execute(self, query_type: str, params: Dict[str, Any] = None) -> ToolResult:
        """데이터베이스 쿼리 실행"""
        start_time = time.time()
        params = params or {}
        
        print(f"💾 DB 쿼리 실행: {query_type}, 파라미터: {params}")
        
        try:
            cursor = self.connection.cursor()
            
            # 쿼리 타입별 처리
            if query_type == "user_info":
                result = self._query_user_info(cursor, params)
            elif query_type == "order_history":
                result = self._query_order_history(cursor, params)
            elif query_type == "product_info":
                result = self._query_product_info(cursor, params)
            elif query_type == "analytics":
                result = self._query_analytics(cursor, params)
            else:
                raise ValueError(f"지원하지 않는 쿼리 타입: {query_type}")
            
            execution_time = time.time() - start_time
            record_count = len(result) if isinstance(result, list) else 1
            
            print(f"✅ DB 쿼리 완료: {record_count}건 조회")
            
            return self._create_result(
                success=True,
                data={
                    "query_type": query_type,
                    "result": result,
                    "record_count": record_count
                },
                execution_time=execution_time
            )
            
        except Exception as e:
            execution_time = time.time() - start_time
            error_msg = f"DB 쿼리 실패: {str(e)}"
            print(f"❌ {error_msg}")
            
            return self._create_result(
                success=False,
                error=error_msg,
                execution_time=execution_time
            )
    
    def _query_user_info(self, cursor, params: Dict[str, Any]) -> List[Dict[str, Any]]:
        """사용자 정보 조회"""
        if 'user_id' in params:
            cursor.execute('SELECT * FROM users WHERE user_id = ?', (params['user_id'],))
        elif 'username' in params:
            cursor.execute('SELECT * FROM users WHERE username = ?', (params['username'],))
        elif 'status' in params:
            cursor.execute('SELECT * FROM users WHERE status = ?', (params['status'],))
        else:
            cursor.execute('SELECT * FROM users ORDER BY created_at DESC LIMIT 10')
        
        return [dict(row) for row in cursor.fetchall()]
    
    def _query_order_history(self, cursor, params: Dict[str, Any]) -> List[Dict[str, Any]]:
        """주문 내역 조회"""
        base_query = '''
            SELECT o.*, u.username, p.name as product_name, p.category, p.price
            FROM orders o
            JOIN users u ON o.user_id = u.user_id
            JOIN products p ON o.product_id = p.product_id
        '''
        
        if 'user_id' in params:
            query = base_query + ' WHERE o.user_id = ? ORDER BY o.order_date DESC'
            cursor.execute(query, (params['user_id'],))
        elif 'status' in params:
            query = base_query + ' WHERE o.status = ? ORDER BY o.order_date DESC LIMIT 20'
            cursor.execute(query, (params['status'],))
        else:
            query = base_query + ' ORDER BY o.order_date DESC LIMIT 20'
            cursor.execute(query)
        
        return [dict(row) for row in cursor.fetchall()]
    
    def _query_product_info(self, cursor, params: Dict[str, Any]) -> List[Dict[str, Any]]:
        """상품 정보 조회"""
        if 'product_id' in params:
            cursor.execute('SELECT * FROM products WHERE product_id = ?', (params['product_id'],))
        elif 'category' in params:
            cursor.execute('SELECT * FROM products WHERE category = ? ORDER BY name', (params['category'],))
        elif 'low_stock' in params and params['low_stock']:
            cursor.execute('SELECT * FROM products WHERE stock < 20 ORDER BY stock')
        else:
            cursor.execute('SELECT * FROM products ORDER BY name LIMIT 10')
        
        return [dict(row) for row in cursor.fetchall()]
    
    def _query_analytics(self, cursor, params: Dict[str, Any]) -> Dict[str, Any]:
        """분석 데이터 조회"""
        analytics = {}
        
        # 기본 통계
        cursor.execute('SELECT COUNT(*) as total_users FROM users WHERE status = "active"')
        analytics['active_users'] = cursor.fetchone()[0]
        
        cursor.execute('SELECT COUNT(*) as total_orders FROM orders')
        analytics['total_orders'] = cursor.fetchone()[0]
        
        cursor.execute('SELECT SUM(total_amount) as total_revenue FROM orders WHERE status = "completed"')
        analytics['total_revenue'] = cursor.fetchone()[0] or 0.0
        
        cursor.execute('SELECT COUNT(*) as pending_orders FROM orders WHERE status = "pending"')
        analytics['pending_orders'] = cursor.fetchone()[0]
        
        # 인기 상품 TOP 3
        cursor.execute('''
            SELECT p.name, SUM(o.quantity) as total_sold, SUM(o.total_amount) as revenue
            FROM orders o
            JOIN products p ON o.product_id = p.product_id
            WHERE o.status = "completed"
            GROUP BY p.product_id
            ORDER BY total_sold DESC
            LIMIT 3
        ''')
        analytics['top_products'] = [dict(row) for row in cursor.fetchall()]
        
        # 카테고리별 판매 현황
        cursor.execute('''
            SELECT p.category, COUNT(*) as order_count, SUM(o.total_amount) as category_revenue
            FROM orders o
            JOIN products p ON o.product_id = p.product_id
            WHERE o.status = "completed"
            GROUP BY p.category
            ORDER BY category_revenue DESC
        ''')
        analytics['category_performance'] = [dict(row) for row in cursor.fetchall()]
        
        # 최근 7일 주문 트렌드 (모의 데이터)
        cursor.execute('''
            SELECT DATE(order_date) as order_date, COUNT(*) as daily_orders
            FROM orders
            WHERE order_date >= date('now', '-7 days')
            GROUP BY DATE(order_date)
            ORDER BY order_date
        ''')
        analytics['daily_trends'] = [dict(row) for row in cursor.fetchall()]
        
        return analytics

# 데이터베이스 도구 테스트
def test_database_tool():
    """데이터베이스 도구 테스트"""
    print("\n🧪 데이터베이스 도구 테스트:")
    
    db_tool = DatabaseTool()
    
    # 다양한 쿼리 테스트
    test_cases = [
        ("user_info", {}, "전체 사용자 조회"),
        ("user_info", {"user_id": 1}, "특정 사용자 조회"),
        ("user_info", {"status": "active"}, "활성 사용자 조회"),
        ("product_info", {"category": "도서"}, "도서 카테고리 상품 조회"),
        ("product_info", {"low_stock": True}, "재고 부족 상품 조회"),
        ("order_history", {"user_id": 1}, "특정 사용자 주문 내역"),
        ("order_history", {"status": "pending"}, "대기중 주문 조회"),
        ("analytics", {}, "분석 데이터 조회")
    ]
    
    for query_type, params, description in test_cases:
        print(f"\n📊 테스트: {description}")
        print(f"   쿼리: {query_type}, 파라미터: {params}")
        
        result = db_tool.execute(query_type=query_type, params=params)
        
        if result.success:
            data = result.data
            print(f"   ✅ 성공: {data['record_count']}건 조회")
            print(f"   ⏱️ 실행시간: {result.execution_time:.3f}초")
            
            # 결과 미리보기
            if query_type == "analytics":
                analytics_result = data['result']
                print(f"   📈 활성 사용자: {analytics_result['active_users']}명")
                print(f"   📦 총 주문: {analytics_result['total_orders']}건")
                print(f"   💰 총 매출: ${analytics_result['total_revenue']:.2f}")
                print(f"   ⏳ 대기 주문: {analytics_result['pending_orders']}건")
                
                if analytics_result['top_products']:
                    print("   🏆 인기 상품:")
                    for i, product in enumerate(analytics_result['top_products'], 1):
                        print(f"     {i}. {product['name']} (판매: {product['total_sold']}개)")
            
            elif isinstance(data['result'], list) and data['result']:
                # 첫 번째 결과만 미리보기
                first_result = data['result'][0]
                print(f"   📋 첫 번째 결과: {dict(list(first_result.items())[:3])}...")
        else:
            print(f"   ❌ 실패: {result.error}")

# 테스트 실행
test_database_tool()

## 6. Tool Manager 및 Function Calling 구현

모든 도구를 관리하고 OpenAI Function Calling을 구현합니다.

In [ ]:
@dataclass
class ToolCall:
    """Tool 호출 정보"""
    tool_name: str
    parameters: Dict[str, Any]
    call_id: str
    timestamp: datetime
    result: Optional[ToolResult] = None

class ToolManager:
    """Tool 관리 및 실행 시스템"""
    
    def __init__(self):
        self.tools: Dict[str, BaseTool] = {}
        self.call_history: List[ToolCall] = []
        
        # 기본 도구들 등록
        self._register_default_tools()
        
        print(f"🛠️ Tool Manager 초기화 완료 - 등록된 도구: {len(self.tools)}개")
    
    def _register_default_tools(self):
        """기본 도구들 등록"""
        try:
            # 날씨 도구
            weather_tool = WeatherTool()
            self.register_tool(weather_tool)
            
            # 검색 도구
            search_tool = SearchTool()
            self.register_tool(search_tool)
            
            # 데이터베이스 도구
            db_tool = DatabaseTool()
            self.register_tool(db_tool)
            
            print("✅ 기본 도구 등록 완료")
            
        except Exception as e:
            print(f"❌ 기본 도구 등록 실패: {e}")
    
    def register_tool(self, tool: BaseTool):
        """도구 등록"""
        self.tools[tool.name] = tool
        print(f"🔧 도구 등록: {tool.name}")
    
    def get_openai_functions(self) -> List[Dict[str, Any]]:
        """OpenAI Function Calling용 함수 스펙 목록 반환"""
        return [tool.get_openai_function_spec() for tool in self.tools.values()]
    
    def execute_tool(self, tool_name: str, parameters: Dict[str, Any]) -> ToolResult:
        """도구 실행"""
        call_id = str(uuid.uuid4())
        call_timestamp = datetime.now()
        
        print(f"🔧 도구 실행: {tool_name} (호출 ID: {call_id[:8]}...)")
        
        # ToolCall 객체 생성
        tool_call = ToolCall(
            tool_name=tool_name,
            parameters=parameters,
            call_id=call_id,
            timestamp=call_timestamp
        )
        
        try:
            if tool_name not in self.tools:
                raise ValueError(f"등록되지 않은 도구: {tool_name}")
            
            tool = self.tools[tool_name]
            result = tool.execute(**parameters)
            tool_call.result = result
            
            # 호출 히스토리에 저장
            self.call_history.append(tool_call)
            
            # 히스토리 크기 제한 (최근 50개만 유지)
            if len(self.call_history) > 50:
                self.call_history = self.call_history[-50:]
            
            status = "✅ 성공" if result.success else "❌ 실패"
            print(f"🔧 도구 실행 완료: {tool_name} - {status}")
            
            return result
            
        except Exception as e:
            error_result = ToolResult(
                success=False,
                data=None,
                error=str(e),
                execution_time=0.0,
                metadata={'tool_name': tool_name, 'call_id': call_id}
            )
            
            tool_call.result = error_result
            self.call_history.append(tool_call)
            
            print(f"❌ 도구 실행 실패: {tool_name} - {str(e)}")
            
            return error_result
    
    def get_call_history(self, limit: int = 10) -> List[Dict[str, Any]]:
        """도구 호출 히스토리 조회"""
        recent_calls = self.call_history[-limit:] if limit > 0 else self.call_history
        
        return [
            {
                'call_id': call.call_id,
                'tool_name': call.tool_name,
                'parameters': call.parameters,
                'timestamp': call.timestamp.isoformat(),
                'success': call.result.success if call.result else False,
                'execution_time': call.result.execution_time if call.result else 0.0,
                'error': call.result.error if call.result and not call.result.success else None
            }
            for call in recent_calls
        ]

# Function Calling 챗봇
class FunctionCallingChatbot:
    """OpenAI Function Calling 기능을 활용한 챗봇"""
    
    def __init__(self):
        self.client = OpenAI(api_key=config.llm.openai_api_key)
        self.tool_manager = ToolManager()
        self.conversation_history: List[Dict[str, str]] = []
        
        print("🤖 Function Calling 챗봇 초기화 완료")
    
    def chat(self, user_message: str) -> Dict[str, Any]:
        """사용자 메시지 처리 및 응답 생성"""
        start_time = time.time()
        print(f"\n💬 Function Calling 챗봇 처리 시작")
        print(f"👤 사용자: {user_message}")
        
        try:
            # 대화 히스토리에 사용자 메시지 추가
            self.conversation_history.append({
                "role": "user",
                "content": user_message
            })
            
            # OpenAI API 호출 (Function Calling 포함)
            messages = self._build_messages()
            functions = self.tool_manager.get_openai_functions()
            
            print(f"🔧 사용 가능한 함수: {len(functions)}개")
            
            response = self.client.chat.completions.create(
                model=config.llm.openai_model,
                messages=messages,
                functions=functions if functions else None,
                function_call="auto" if functions else None,
                temperature=config.llm.temperature
            )
            
            # 응답 처리
            result = self._process_response(response, user_message)
            
            processing_time = time.time() - start_time
            result['processing_time'] = processing_time
            
            print(f"✅ Function Calling 처리 완료 - 시간: {processing_time:.2f}초")
            
            return result
            
        except Exception as e:
            processing_time = time.time() - start_time
            error_msg = f"Function Calling 처리 실패: {str(e)}"
            print(f"❌ {error_msg}")
            
            return {
                'success': False,
                'response': "죄송합니다. 처리 중 오류가 발생했습니다.",
                'error': error_msg,
                'processing_time': processing_time
            }
    
    def _build_messages(self) -> List[Dict[str, str]]:
        """대화 메시지 구성"""
        system_message = {
            "role": "system",
            "content": """당신은 다양한 도구를 활용할 수 있는 AI 어시스턴트입니다.
            
사용 가능한 도구:
- get_weather: 날씨 정보 조회 (도시명 필요)
- web_search: 인터넷 검색 (검색어 필요)
- query_database: 데이터베이스 조회 (쿼리 타입 필요)

사용자의 요청에 따라 적절한 도구를 사용하여 정확하고 유용한 정보를 제공하세요.
도구 사용이 필요하지 않은 일반적인 질문에는 직접 답변하세요.
한국어로 친근하고 도움이 되는 답변을 제공하세요."""
        }
        
        return [system_message] + self.conversation_history[-8:]  # 최근 8개 메시지만 유지
    
    def _process_response(self, response, user_message: str) -> Dict[str, Any]:
        """OpenAI 응답 처리"""
        message = response.choices[0].message
        
        # Function Call이 있는지 확인
        if hasattr(message, 'function_call') and message.function_call:
            print(f"🔧 Function Call 감지: {message.function_call.name}")
            return self._handle_function_call(message, user_message)
        else:
            # 일반 응답 처리
            ai_response = message.content
            self.conversation_history.append({
                "role": "assistant",
                "content": ai_response
            })
            
            print(f"🤖 AI 응답: {ai_response[:100]}...")
            
            return {
                'success': True,
                'response': ai_response,
                'function_calls': [],
                'tokens_used': response.usage.total_tokens
            }
    
    def _handle_function_call(self, message, user_message: str) -> Dict[str, Any]:
        """Function Call 처리"""
        function_call = message.function_call
        function_name = function_call.name
        
        try:
            # 함수 파라미터 파싱
            function_args = json.loads(function_call.arguments)
            
            print(f"⚙️ Function Call 실행: {function_name}({function_args})")
            
            # 도구 실행
            tool_result = self.tool_manager.execute_tool(function_name, function_args)
            
            # Function Call 결과를 기반으로 추가 응답 생성
            function_result_message = {
                "role": "function",
                "name": function_name,
                "content": json.dumps(tool_result.data if tool_result.success else {"error": tool_result.error})
            }
            
            # 대화 히스토리에 추가
            self.conversation_history.append({
                "role": "assistant",
                "content": None,
                "function_call": {
                    "name": function_name,
                    "arguments": function_call.arguments
                }
            })
            self.conversation_history.append(function_result_message)
            
            # 최종 응답 생성
            print("📝 최종 응답 생성 중...")
            final_response = self.client.chat.completions.create(
                model=config.llm.openai_model,
                messages=self._build_messages(),
                temperature=config.llm.temperature
            )
            
            final_message = final_response.choices[0].message.content
            self.conversation_history.append({
                "role": "assistant",
                "content": final_message
            })
            
            print(f"🤖 최종 응답: {final_message[:100]}...")
            
            return {
                'success': True,
                'response': final_message,
                'function_calls': [{
                    'function_name': function_name,
                    'arguments': function_args,
                    'result': tool_result.data if tool_result.success else None,
                    'error': tool_result.error if not tool_result.success else None,
                    'execution_time': tool_result.execution_time
                }],
                'tokens_used': final_response.usage.total_tokens
            }
            
        except Exception as e:
            print(f"❌ Function Call 처리 실패: {e}")
            
            return {
                'success': False,
                'response': f"도구 실행 중 오류가 발생했습니다: {str(e)}",
                'function_calls': [{
                    'function_name': function_name,
                    'error': str(e)
                }]
            }

# Function Calling 챗봇 테스트
def test_function_calling_chatbot():
    """Function Calling 챗봇 종합 테스트"""
    print("\n🧪 Function Calling 챗봇 종합 테스트:")
    
    chatbot = FunctionCallingChatbot()
    
    # 다양한 시나리오 테스트
    test_messages = [
        "안녕하세요! 어떤 도구를 사용할 수 있나요?",
        "서울 날씨 알려주세요",
        "Python 프로그래밍에 대해 검색해주세요",
        "데이터베이스에서 사용자 정보를 조회해주세요",
        "매출 분석 데이터를 보여주세요"
    ]
    
    for i, message in enumerate(test_messages, 1):
        print(f"\n{'='*60}")
        print(f"🧪 테스트 {i}: {message}")
        print(f"{'='*60}")
        
        result = chatbot.chat(message)
        
        if result['success']:
            print(f"\n🤖 AI 응답:")
            print(result['response'])
            
            # Function Call 정보 표시
            if result.get('function_calls'):
                print(f"\n🔧 실행된 도구:")
                for call in result['function_calls']:
                    status = "✅" if not call.get('error') else "❌"
                    print(f"  {status} {call['function_name']}")
                    print(f"     파라미터: {call['arguments']}")
                    print(f"     실행시간: {call['execution_time']:.2f}초")
                    if call.get('error'):
                        print(f"     오류: {call['error']}")
            
            print(f"\n📊 메타데이터:")
            print(f"  토큰 사용: {result.get('tokens_used', 'N/A')}")
            print(f"  처리 시간: {result['processing_time']:.2f}초")
        else:
            print(f"\n❌ 오류: {result.get('error')}")
        
        # 테스트 간 간격
        time.sleep(1)
    
    # 도구 호출 히스토리 출력
    print(f"\n📊 도구 호출 히스토리:")
    call_history = chatbot.tool_manager.get_call_history()
    
    for call in call_history:
        status = "✅" if call['success'] else "❌"
        print(f"  {status} {call['tool_name']} - {call['execution_time']:.2f}초")
        print(f"     시간: {call['timestamp'][11:19]}")
        if call['error']:
            print(f"     오류: {call['error']}")

# 테스트 실행
test_function_calling_chatbot()

## 7. 실습 정리 및 확장 아이디어

5차시 실습의 핵심 내용을 정리하고 실무에서의 확장 가능성을 제시합니다.

In [ ]:
def print_lesson5_summary():
    """5차시 실습 요약"""
    print("""\n📚 5차시 실습 요약: 외부 연동 & 실시간 데이터 처리
    
🎯 달성한 학습 목표:
✅ OpenAI Function Calling 완전 구현
✅ 외부 API 연동 (날씨, 검색)
✅ 데이터베이스 실시간 연동
✅ Circuit Breaker 패턴 적용
✅ 재시도 로직 및 에러 핸들링
✅ Tool 시스템 아키텍처 구축

🔧 핵심 구현 사항:
• BaseTool: 모든 도구의 기본 클래스
• WeatherTool: 날씨 API 연동 (OpenWeatherMap 호환)
• SearchTool: 웹 검색 (DuckDuckGo API 활용)
• DatabaseTool: SQLite/PostgreSQL 연동
• ToolManager: 도구 등록 및 실행 관리
• FunctionCallingChatbot: OpenAI Function Calling 통합
• CircuitBreaker: 외부 서비스 장애 대응

📊 구현 특징:
• 모듈화된 Tool 아키텍처
• 강력한 에러 핸들링 및 복구
• 실시간 데이터 처리
• 확장 가능한 도구 시스템
• 상세한 로깅 및 모니터링

🚀 실무 응용 사례:
1. 고객 서비스 봇
   - 주문 조회, 배송 추적, 재고 확인
   - CRM 시스템 연동
   - 실시간 고객 데이터 조회

2. 업무 자동화 봇
   - 이메일 발송, 캘린더 관리
   - 프로젝트 관리 도구 연동
   - 문서 생성 및 업데이트

3. 데이터 분석 봇
   - 실시간 대시보드 생성
   - 데이터베이스 쿼리 자동화
   - 리포트 자동 생성

4. IoT 제어 봇
   - 스마트 홈 기기 제어
   - 센서 데이터 모니터링
   - 자동화 시나리오 실행

🔗 연동 가능한 서비스:
• 클라우드 서비스: AWS, GCP, Azure
• 데이터베이스: PostgreSQL, MySQL, MongoDB
• API 서비스: REST, GraphQL, gRPC
• 메시징: Slack, Discord, Teams
• 결제: Stripe, PayPal
• 소셜미디어: Twitter, Instagram
• 이메일: Gmail, Outlook
• 파일 저장소: Google Drive, Dropbox""")

def suggest_advanced_implementations():
    """고급 구현 아이디어 제안"""
    print("""\n🎓 고급 구현 아이디어:

1. 🔄 비동기 Tool 실행
   - AsyncIO 기반 병렬 도구 실행
   - 스트리밍 결과 처리
   - 대용량 데이터 처리 최적화

2. 🧠 지능형 Tool 선택
   - 컨텍스트 기반 도구 추천
   - 사용 패턴 학습 및 최적화
   - 다단계 Tool Chain 구성

3. 🔒 보안 강화
   - OAuth 2.0 인증 통합
   - API 키 암호화 저장
   - 사용자별 권한 관리
   - 감사 로그 및 추적

4. 📈 성능 모니터링
   - Prometheus 메트릭 수집
   - 실시간 성능 대시보드
   - 자동 스케일링
   - 캐싱 전략 구현

5. 🔧 Tool 개발 SDK
   - 커스텀 Tool 개발 프레임워크
   - 플러그인 시스템
   - 자동 문서 생성
   - 테스트 자동화

6. 🌐 마이크로서비스 아키텍처
   - Tool을 독립 서비스로 분리
   - 서비스 디스커버리
   - 로드 밸런싱
   - 분산 트레이싱""")

def demonstrate_real_world_scenario():
    """실제 비즈니스 시나리오 시연"""
    print("\n🏢 실제 비즈니스 시나리오 시연:")
    print("고객 서비스 봇이 복합적 요청을 처리하는 과정\n")
    
    # 고객 서비스 시나리오
    chatbot = FunctionCallingChatbot()
    
    customer_scenarios = [
        "사용자 john_doe의 최근 주문 내역을 확인해주세요",
        "재고가 부족한 상품들을 알려주세요",
        "전체 매출 분석 데이터를 보여주세요"
    ]
    
    for i, scenario in enumerate(customer_scenarios, 1):
        print(f"📋 시나리오 {i}: {scenario}")
        print("-" * 50)
        
        result = chatbot.chat(scenario)
        
        if result['success']:
            print(f"🤖 봇 응답: {result['response'][:200]}...\n")
            
            if result.get('function_calls'):
                for call in result['function_calls']:
                    print(f"⚙️ 실행된 도구: {call['function_name']}")
                    print(f"⏱️ 실행 시간: {call['execution_time']:.2f}초")
                    print(f"✅ 상태: {'성공' if not call.get('error') else '실패'}")
        else:
            print(f"❌ 오류: {result.get('error')}")
        
        print("\n" + "="*60 + "\n")

# 실습 요약 및 시연 실행
print_lesson5_summary()
suggest_advanced_implementations()

# 실제 시나리오 시연
demonstrate_real_world_scenario()

# 실습 완료 체크리스트
print("\n" + "="*60)
print("🎉 5차시 실습이 성공적으로 완료되었습니다!")
print("📝 다음은 6차시: 성능 최적화 & 모니터링입니다.")
print("="*60)

checklist = [
    "✅ Tool 베이스 클래스 및 아키텍처 구현",
    "✅ 날씨 API 연동 도구 완성",
    "✅ 웹 검색 도구 구현",
    "✅ 데이터베이스 연동 도구 완성",
    "✅ Tool Manager 및 히스토리 관리",
    "✅ OpenAI Function Calling 통합",
    "✅ Circuit Breaker 패턴 적용",
    "✅ 재시도 로직 및 에러 핸들링",
    "✅ 실제 비즈니스 시나리오 테스트"
]

print("\n📋 실습 체크리스트:")
for item in checklist:
    print(f"  {item}")

print("\n🔜 다음 실습 예고:")
print("  • 응답 캐싱 시스템 (Redis)")
print("  • 비동기 처리 및 병렬 실행")
print("  • 토큰 사용량 최적화")
print("  • 실시간 모니터링 시스템")
print("  • 성능 메트릭 수집 및 분석")